In [8]:
import os
import re
import pandas as pd
import urllib
from nltk import SnowballStemmer
from gensim.models import Word2Vec
#import langdetect
import tika
import time
from tika import parser
import pickle

# Raw Articles pdf Data
Convert pdf to text using Tika apache server 

In [9]:
# specify the target data folder
target_dir = 'C:/Thesis/Data/Journal_of_applied_psychology'

In [10]:
# prep pdf extraction
pdf_files = []
wrd_files = []

for f in os.listdir(target_dir):
    if f.endswith(".pdf") | f.endswith(".PDF"):
        thispdf = os.path.join(target_dir, f)
        pdf_files = pdf_files + [thispdf]
    if f.endswith(".doc") | f.endswith(".docx") | f.endswith(".DOC") | f.endswith(".DOCX"):
        thiswrd = os.path.join(target_dir, f)
        wrd_files = wrd_files + [thiswrd]

In [11]:
port = 4321 # port to use for Tika server (chosen arbitrarily)
server_str = 'http://localhost:' + str(port)
tika_path  = 'C:\Software\ tika-server-1.23.jar'
tika_run   = 'java -jar' + tika_path + ' --port ' + str(port)
print('Command to start Tika:')
print(tika_run)
print('---')
print('Number of pdf files: ' + str(len(pdf_files)))
print('Number of word files: ' + str(len(wrd_files)))

Command to start Tika:
java -jarC:\Software\ tika-server-1.23.jar --port 4321
---
Number of pdf files: 574
Number of word files: 0


# pdf metadata

In [12]:
# process the PDF files, store text in memory
tika.TikaClientOnly = True 

pdf_metadata = [parser.from_file(t, server_str, xmlContent=False)["metadata"] for t in (pdf_files + wrd_files)]

2020-05-24 16:43:13,172 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to C:\Users\ICTO-EB\AppData\Local\Temp\tika-server.jar.
2020-05-24 16:43:17,911 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to C:\Users\ICTO-EB\AppData\Local\Temp\tika-server.jar.md5.
2020-05-24 16:43:18,783 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [13]:
# save data metadata
pickle.dump(pdf_metadata, open( "C:/Thesis/Data/save/Journal_of_applied_psychology/metadata/metadata_all.p", "wb" ) )

# pdf content

In [ ]:
# process the PDF files, store text in memory
tika.TikaClientOnly = True 

#pdf_text = [parser.from_file(t, server_str, xmlContent=False)["content"] for t in (pdf_files + wrd_files)]

# step :1 Load pdf content

In [4]:
import pickle
#load the text data
pdf_data = pickle.load(open("C:/Thesis/Data/save/Journal_of_applied_psychology/save_alltextData.p", "rb"))


# step-2 convert it to lowercase

In [5]:
# convert pdf data to lowercase
pdf_text=[]
for i in pdf_data:
    if i is not None:
         pdf_text.append(i.lower())

# step-3 remove reference

In [7]:
# remove references section 
import re
pdf_ref = [re.sub(r"(?is)\nreferences\n.+", "", f) for f in pdf_text]

# step-4 load metadata

In [14]:
import pickle
#load the metadata
pdf_metadata = pickle.load(open("C:/Thesis/Data/save/Journal_of_applied_psychology/metadata/metadata_all.p", "rb"))

# step-5 get metadata : author, subject, title

In [20]:
pdf_title = []
pdf_Author = []
pdf_subject = []

# it consists of few title and subjects 
for i in pdf_metadata:
    temp =""
    auth_temp = ""
    subj_temp = ""
    try:
        temp = i['title']
        auth_temp = i['Author']
        subj_temp = i['subject']
     
        pdf_title.append(temp.lower()) 
        pdf_Author.append(auth_temp) # got error while converting it into lowercases. lets remove it from inital dataset
        pdf_subject.append(subj_temp.lower())
       
    except:
        #print("An exception occurred" )
        pdf_title.append(temp.lower())
        pdf_Author.append(auth_temp)
        pdf_subject.append(subj_temp.lower())

In [29]:
pdf_Author

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'jantonak',
 '',
 'KSU',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Peter A. Bamberger, Jaclyn Koopmann, Mo Wang, Mary Larimer, Inbal Nahum-Shani, Irene Geisner, Samuel B. Bacharach',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Administrator',
 'Administrator',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Design Unit London School of Economics',
 'Mark N. Bing',
 '',
 '',
 '',
 '',
 'edwardsj',
 '',
 '',
 'lbs01jc',
 'Neil Stewart',
 'Design Unit London School of Economics',
 '',
 '',
 'Boyce',
 'Jennifer Bragger, Donald Bragger, Donald A. Hantula, Jean Kirnan, and Eugene Kutcher',
 '',
 '',
 '',
 '',
 '',
 'vb1',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Jonathan Cottrell',
 'IT Services',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'bdejbv',
 '',
 '',
 '',
 '',
 '',
 '',

In [30]:
# pre-process author
import re
pdf_auth = []

for j in pdf_Author:
    k = re.sub(' and ', '', str(j))
    k = re.sub('[!;&$]', '', k)
    k = re.sub('[0-9]+', '', k)
    k = re.sub( r'\b\w{1,3}\b', '', k)
    k = re.sub( r'-', ' ', k)
#     pdf_p = [re.sub(' +', ' ', f) for f in pdf_p]
    k = re.sub(' +', ',', k)
    k = k.split(",")
   
    pdf_auth.append(k)
    

In [ ]:
# not required
def eliminate_method_result(method, discussion):
    
    import re    
    start = method
    end = discussion
    #start of pattern, followed by any number of times 'any character OR a newline' and terminated by the end pattern.
    pattern = start + '.*'+'(.|\n)*'+end
    pdf_method_dis = [re.sub(pattern, '', f) for f in pdf_text]

    return pdf_method_dis

In [ ]:
# not required
def eliminate_method_result_ref(method, dis, ref):
    
    import re

    start = method
    end = dis
    
    #start of pattern, followed by any number of times 'any character OR a newline' and terminated by the end pattern.
    pattern = start + '.*'+'(.|\n)*'+end
    pdf_method_dis = [re.sub(pattern, '', f) for f in pdf_text]
    
    start_ref = ref
    pattern_ref = start_ref +'(.|\n)*$'

    pdf_ref= [re.sub(pattern_ref, '', f) for f in pdf_method_dis]
    
    return pdf_ref

#  Pre-processing
Regular expression is used to clean up the pre-processed data and saved as a Master data

In [33]:
import re
pdf_p= [re.sub(r"-\n(\n)*", "", t)for t in pdf_ref]

pdf_p= [re.sub(r"\n(\n)*", "\n", t)for t in pdf_p]

pdf_p= [re.sub(r"\n", " ", t)for t in pdf_p]
# remove text with [] eg citation
pdf_p = [re.sub(r"\[[^)]*\]", "", t)for t in pdf_p]
#Remove numbers
pdf_p = [re.sub('[0-9]+', '', f) for f in pdf_p]
#pdf_p = [re.sub("r[\S]+\.(net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?", " ", t)for t in pdf_p]
pdf_p = [re.sub('[!@/%“”‘:#©β<>+=δχ*&$]', ' ', f) for f in pdf_p]
pdf_p = [re.sub('’s','', f) for f in pdf_p]
pdf_p = [re.sub('’re','', f) for f in pdf_p]
pdf_p = [re.sub('’t','', f) for f in pdf_p]
pdf_p = [re.sub('’ve','', f) for f in pdf_p]
pdf_p = [re.sub('’ll','', f) for f in pdf_p]
pdf_p = [re.sub('’m','', f) for f in pdf_p]

pdf_p = [re.sub('[:()-]', ' ', f) for f in pdf_p]
pdf_p = [re.sub('[\.!?]+ ', 'XYZXYZ', t) for t in pdf_p]
# \w will match alphanumeric characters and underscores
# [^\w] will match anything that's not alphanumeric or underscore
pdf_p = [re.sub(r'[^\w]', ' ', f) for f in pdf_p]
pdf_p = [re.sub('XYZXYZ', '.', f) for f in pdf_p]
pdf_p = [re.sub(' +', ' ', f) for f in pdf_p]
# Replace multiple dots with one dot
pdf_p = [re.sub('\.\.+',".", f) for f in pdf_p]
pdf_p = [re.sub(r'\b\w{1,3}\b',"", f) for f in pdf_p]
pdf_p = [re.sub(' +', ' ', f) for f in pdf_p]

pdf_p = [re.sub("^\s+","", f) for f in pdf_p]# remove the front space
pdf_p = [re.sub("\s+\Z","", f) for f in pdf_p]#remove the back space

In [34]:
pdf_p[2]

'empower empower your sales force. empirical examination influence leadership empowerment behavior customer satisfaction performance michael ahearne university houston john mathieu adam rapp university connecticut this research focuses impact leadership empowerment behavior customer service satisfaction sales performance mediated salespeople self efficacy adaptability.moreover authors propose interactive relationship whereby will differentially effective function employees empowerment readiness. authors hypotheses tested using survey data from sample salespeople pharmaceutical field along with external ratings satisfaction from customers archival sales performance information.results indicated that contrary popular belief employees with levels product industry knowledge experience benefit most from leadership behaviors that empowering whereas high knowledge experienced employees reap clear benefit. authors conclude with directions future research application.keywords empowerment experi

In [37]:
# author list extracted from metedata

author_list = ['jantonak', 'peter','bamberger','jaclyn', 'koopmann', 'wang', 'mary', 'larimer', 'inbal', 'nahum', 'shani', 'irene', 'geisner', 'samuel', 'bacharach']
author_list += ['administrator', 'administrator', 'design', 'unit', 'london', 'school', 'economics', 'mark', 'bing', 'edwardsj']
author_list += ['lbsjc', 'neil', 'stewart', 'design', 'unit', 'london', 'school', 'economics', 'boyce', 'jennifer', 'bragger', 'donald']
author_list += ['bragger', 'donald', 'hantula', 'jean', 'kirnan', 'eugene', 'kutcher', 'jonathan', 'cottrell', 'services', 'bdejbv']
author_list += ['doerr', 'kenneth', 'howard', 'freed', 'tali', 'mitchell', 'terence', 'schriesheim', 'chester', 'zhou', 'xiaohua']
author_list += ['tracy', 'dawn', 'iacobucci', 'silke', 'eisenbeiss', 'sabine','boerner', 'daan', 'knippenberg', 'kendal', 'kelly']
author_list += ['frese', 'charlotte', 'fritz',  'sabine', 'sonnentag', 'michelep', 'mark', 'griffin', 'sharon', 'parker', 'claire']
author_list += ['mason', 'brian', 'christopher', 'gunia', 'egerber', 'staff', 'yukun','johnsonr', 'mike', 'design', 'unit', 'london']
author_list += ['school', 'economics', 'florian', 'kunze',  'anneloes', 'raes',  'heike', 'bruch', 'jonas', 'snotosoehardjo', 'user']
author_list += ['sandy', 'limkenneth','lbruth', 'lindsey', 'avery', 'dawson', 'king','meyers','kristin','madrid']
author_list += ['totterdell', 'niven', '.barros','eganp']

author_list += ['alyssa', 'mcgonagle', 'gwenith', 'fisher', 'janet', 'barnes', 'farrell', 'james', 'grosch', 'mwalker', 'malpass']
author_list += ['mikolon',  'sven', 'dyang', 'hliao', 'design', 'unit', 'london', 'school', 'economics', 'roni', 'reiter']
author_list += ['palmonamerican', 'institutes', 'research', 'design', 'unit', 'london', 'school', 'economics', 'ashleigh', 'shelby']
author_list += ['rosette', 'julian', 'gamble', 'design', 'unit', 'london', 'school', 'economics', 'sitzmann',  'traci', 'sabine']
author_list += ['sonnentag', 'sabine', 'sonnentag',  'fred', 'zijlstra', 'sabine', 'sonnentag',  'carmen', 'binnewies',  'mojza']
author_list += ["sabine", "sonnentag", "carmen", "binnewies"]
   
author_list += [ "mojza", 'sabine', 'sonnentag', 'mojza', 'evangelia', 'demerouti', 'arnold', 'bakker', 'unsworth',  'klmcneill']
author_list += ['jeff', 'vancouver', 'sozialpsychologie', 'tiffany', 'henry', 'design', 'unit', 'london', 'school', 'economics', 'gyeo']
author_list += ['yoshikawa',  'katsuhhikowu',  'chia', 'hueilee',  'hyun', 'jung', 'jantonak', 'peter', 'jaclyn', 'koopmann', 'wang']
author_list += ['mary', 'larimer', 'inbal', 'nahum', 'shani', 'irene', 'geisner', 'samuel', 'bacharach', 'administrator', 'design']
author_list += ['unit', 'london', 'school', 'economics', 'mark', 'bing', 'edwardsj', 'lbsjc', 'neil', 'stewart', 'boyce', 'jennifer']
author_list += ['bragger', 'donald', 'hantula', 'jean', 'kirnan', 'eugene', 'kutcher', 'jonathan', 'cottrell', 'services', 'bdejbv']
author_list += ['doerr', 'kenneth', 'howard', 'freed', 'tali', 'mitchell', 'terence', 'schriesheim', 'chester', 'zhou', 'xiaohua']
author_list += ['tracy', 'dawn', 'iacobucci', 'silke', 'eisenbeiss', 'sabine', 'boerner', 'daan', 'knippenberg', 'kendal', 'kelly']
author_list += ['frese', 'charlotte', 'fritz',  'sabine', 'sonnentag', 'michelep', 'mark', 'griffin', 'sharon', 'parker', 'claire']
author_list += ['mason', 'brian', 'christopher', 'gunia', 'egerber', 'staff', 'yukun','johnsonr', 'mike', 'florian', 'kunze']
author_list += ['anneloes', 'raes',  'heike', 'bruch', 'jonas', 'snotosoehardjo', 'user', 'sandy', 'limkenneth','lbruth', 'lindsey']
author_list += ['avery', 'dawson', 'king', 'meyers',  'kristin','madrid', 'totterdell', 'niven', '.barros','eganp', 'alyssa']
author_list += ['mcgonagle', 'gwenith', 'fisher', 'janet', 'barnes', 'farrell', 'james', 'grosch', 'mwalker', 'malpass', 'mikolon'] 
author_list += ['sven', 'dyang', 'hliao', 'roni', 'reiter', 'palmonamerican', 'institutes', 'research', 'ashleigh', 'shelby', 'rosette']
author_list += ['julian', 'gamble', 'sitzmann',  'traci', 'sabine', 'sonnentag', 'sabine', 'sonnentag',  'fred', 'zijlstra', 'sabine']
author_list += ['sonnentag',  'carmen', 'binnewies',  'mojza', "sabine", "sonnentag", "carmen", "binnewies", "mojza", 'sabine']
author_list += ['mojza', 'evangelia', 'demerouti', 'arnold', 'bakker', 'unsworth',  'klmcneill','jeff', 'vancouver']
author_list += ['sozialpsychologie', 'tiffany', 'henry', 'gyeo', 'yoshikawa', 'katsuhhikowu', 'chia', 'hueilee',  'hyun', 'jung']

In [38]:
# Function to filter out the stopwords and authors names
def filterWords(msg, words):
    m = msg.split(' ')
    words_end = [w + '.' for w in words]
    filtered_words = list(filter(lambda word: word not in words, m))
    filtered_words = [(w if w not in words_end else '.') for w in filtered_words]
    result = ' '.join(filtered_words)
    return(result)

In [39]:
data_proc_auth = [filterWords(f, author_list) for f in pdf_p]

In [42]:
data_proc_auth[2]

'empower empower your sales force. empirical examination influence leadership empowerment behavior customer satisfaction performance michael ahearne university houston john mathieu adam rapp university connecticut this focuses impact leadership empowerment behavior customer service satisfaction sales performance mediated salespeople self efficacy adaptability.moreover authors propose interactive relationship whereby will differentially effective function employees empowerment readiness. authors hypotheses tested using survey data from sample salespeople pharmaceutical field along with external ratings satisfaction from customers archival sales performance information.results indicated that contrary popular belief employees with levels product industry knowledge experience benefit most from leadership behaviors that empowering whereas high knowledge experienced employees reap clear benefit. authors conclude with directions future application.keywords empowerment experience salespeople s

In [43]:
stopwords = ['a', 'about', 'above', 'across', 'after', 'afterwards']
stopwords += ['again', 'against', 'all', 'almost', 'alone', 'along']
stopwords += ['already', 'also', 'although', 'always', 'am', 'among']
stopwords += ['amongst', 'amoungst', 'amount', 'an', 'and', 'another']
stopwords += ['any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere']
stopwords += ['are', 'around', 'as', 'at', 'back', 'be', 'became']
stopwords += ['because', 'become', 'becomes', 'becoming', 'been']
stopwords += ['before', 'beforehand', 'behind', 'being', 'below']
stopwords += ['beside', 'besides', 'between', 'beyond', 'bill', 'both']
stopwords += ['bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant']
stopwords += ['co', 'computer', 'con', 'could', 'couldnt', 'cry', 'de']
stopwords += ['describe', 'detail', 'did', 'do', 'done', 'down', 'due']
stopwords += ['during', 'each', 'eg', 'eight', 'either', 'eleven', 'else']
stopwords += ['elsewhere', 'empty', 'enough', 'etc', 'even', 'ever']
stopwords += ['every', 'everyone', 'everything', 'everywhere', 'except']
stopwords += ['few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first']
stopwords += ['five', 'for', 'former', 'formerly', 'forty', 'found']
stopwords += ['four', 'from', 'front', 'full', 'further', 'get', 'give']
stopwords += ['go', 'had', 'has', 'hasnt', 'have', 'he', 'hence', 'her']
stopwords += ['here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers']
stopwords += ['herself', 'him', 'himself', 'his', 'how', 'however']
stopwords += ['hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed']
stopwords += ['interest', 'into', 'is', 'it', 'its', 'itself', 'keep']
stopwords += ['last', 'latter', 'latterly', 'least', 'less', 'ltd', 'made']
stopwords += ['many', 'may', 'me', 'meanwhile', 'might', 'mill', 'mine']
stopwords += ['more', 'moreover', 'most', 'mostly', 'move', 'much']
stopwords += ['must', 'my', 'myself', 'name', 'namely', 'neither', 'never']
stopwords += ['nevertheless', 'next', 'nine', 'no', 'nobody', 'none']
stopwords += ['noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of']
stopwords += ['off', 'often', 'on','once', 'one', 'only', 'onto', 'or']
stopwords += ['other', 'others', 'otherwise', 'our', 'ours', 'ourselves']
stopwords += ['out', 'over', 'own', 'part', 'per', 'perhaps', 'please']
stopwords += ['put', 'rather', 're', 's', 'same', 'see', 'seem', 'seemed']
stopwords += ['seeming', 'seems', 'serious', 'several', 'she', 'should']
stopwords += ['show', 'side', 'since', 'sincere', 'six', 'sixty', 'so']
stopwords += ['some', 'somehow', 'someone', 'something', 'sometime']
stopwords += ['sometimes', 'somewhere', 'still', 'such', 'system', 'take']
stopwords += ['ten', 'than', 'that', 'the', 'their', 'them', 'themselves']
stopwords += ['then', 'thence', 'there', 'thereafter', 'thereby']
stopwords += ['therefore', 'therein', 'thereupon', 'these', 'they']
stopwords += ['thick', 'thin', 'third', 'this', 'those', 'though', 'three']
stopwords += ['three', 'through', 'throughout', 'thru', 'thus', 'to']
stopwords += ['together', 'too', 'top', 'toward', 'towards', 'twelve']
stopwords += ['twenty', 'two', 'un', 'under', 'until', 'up', 'upon']
stopwords += ['us', 'very', 'via', 'was', 'we', 'well', 'were', 'what']
stopwords += ['whatever', 'when', 'whence', 'whenever', 'where']
stopwords += ['whereafter', 'whereas', 'whereby', 'wherein', 'whereupon']
stopwords += ['wherever', 'whether', 'which', 'while', 'whither', 'who']
stopwords += ['whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with']
stopwords += ['within', 'without', 'would', 'yet', 'you', 'your']
stopwords += ['yours', 'yourself', 'yourselves']

In [44]:
# GET RID OF THE STOPWORDS IN TEXTS
data_proc = [filterWords(f, stopwords) for f in data_proc_auth]

# Get country name

In [45]:
import pycountry

country_list = []
for country in pycountry.countries:
    a = (country.name).lower()
    country_list.append(a) 

In [46]:
data_proc_country =[]
for j in data_proc:
    line = j
    for k in country_list:
        line = re.sub(k, '', line)
        
    data_proc_country.append(line)

In [48]:
# data_proc_country[2]

In [49]:
# Authors names are saved already (code is available at author name folder-get name list notebook)
#load authors names
import pickle
author_nam = pickle.load(open("C:/Thesis/Data/save/Master_Data/auth_ref_filter_num_char/auth_Journal_of_applied_psychology.p", "rb"))

In [50]:
# GET REFERENCE LISTS OF STOPWORDS, I.E. WORDS THAT ARE ESSENTIALLY MEANINGLESS
# Get all the list of authors names

from nltk.corpus import stopwords

sw = stopwords.words('english') + author_nam 

#print(sw)

In [51]:
data_proc_1 = [filterWords(f, sw) for f in data_proc_country]  #running

In [52]:
data_proc_2 = [re.sub(' \.+', ' ', f) for f in data_proc_1]
data_proc_2 = [re.sub(' +', ' ', f) for f in data_proc_2]
data_proc_2 = [re.sub('http', '', f) for f in data_proc_2]
data_proc_2 = [re.sub('www', '', f) for f in data_proc_2]
data_proc_2 = [re.sub('iweb', '', f) for f in data_proc_2]
data_proc_2 = [re.sub('email ', '', f) for f in data_proc_2]
data_proc_2 = [re.sub(' +', ' ', f) for f in data_proc_2]


In [53]:
data_proc_2[2]

'empower empower empirical examination influence leadership empowerment customer satisfaction performance university adam university connecticut focuses impact leadership empowerment customer service satisfaction performance mediated salespeople efficacy adaptability.moreover authors propose interactive relationship differentially effective function employees empowerment readiness. authors hypotheses tested using survey salespeople pharmaceutical external ratings satisfaction customers archival performance information.results indicated contrary popular belief employees levels product experience benefit leadership behaviors empowering high experienced employees reap clear benefit. authors conclude directions future application.keywords empowerment experience salespeople efficacy performance century began organizational landscape transformed.advances technology global marketplace nafta union sagging economy factors demanded organizations leaner make practices cost effective closer custom

In [54]:
# save data
pickle.dump(data_proc_2, open( "C:/Thesis/Data/save/Master_Data/MD_1/data_proc_latest/J_of_applied_psy_data_proc.p", "wb" ) )

In [55]:
#import io
with open("C:/Thesis/Data/save/Master_Data/MD_1/data_proc_latest/J_of_applied_psy_data_proc_txt.txt", "w",encoding="utf-8") as outfile:
    for i in range(len(data_proc_2)):
        outstring = ""
        outstring += str(data_proc_2[i])
        outfile.write(outstring)